In [2]:
import pandas as pd
import numpy as np
import pickle as p

In [3]:
data_all = pd.read_csv('../data_all.csv')

# Test with Subset of Data
np.random.seed(43)
item_subset = list(np.random.choice(data_all['Item_ID'].unique(), size=10, replace=False))
data_subset = data_all.loc[data_all['Item_ID'].isin(item_subset)].copy()

# # Test with All Data
# data_subset = data_all.copy()

data_subset.dropna(axis=0, inplace=True)
data_subset.isnull().sum().sum()

0

In [42]:
# item_subset

### `/train`

In [6]:
import time

In [37]:
data_subset.to_csv('data_subset_10items_seed43.csv', index=None)

In [43]:
import requests
import json
from threading import Thread

url = 'https://fillet.azurewebsites.net/train/'
# url = 'http://localhost:5000/train/'

data_subset.to_parquet('data_subset.parquet')
files = {'data': open('data_subset.parquet', 'rb')}

payload = {'cv_acc': True, 'project_id': 'cccc', 'modeltype': 'default'}

Thread(target=requests.post,
       args=(url, ),
       kwargs={
           'data': payload,
           'files': files
       }).start()

time.sleep(10)
files['data'].close()

### `/query_progress`

In [9]:
import requests
import json

url = 'https://fillet.azurewebsites.net/query_progress/'
# url = 'http://localhost:5000/query_progress/'


payload = {'project_id': 'cccc'}

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload,headers=headers,  timeout=3)

print('% Complete:',round(r.json()['pct_complete'],3))
print('CV % Complete:',round(r.json()['cv_progress'],3))
print('CV Done:',r.json()['cv_done'])

% Complete: 100.0
CV % Complete: 100.0
CV Done: 1


### `/batch_query_progress`

In [39]:
import requests
import json

url = 'https://fillet.azurewebsites.net/batch_query_progress/'
# url = 'http://localhost:5000/query_progress/'


payload = {'project_id_ls': ['bbbb','aaaa','cccc']}

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload,headers=headers,  timeout=3)

print(r.json())

{'aaaa': {'cv_done': 1, 'cv_progress': 100.0, 'pct_complete': 100.0, 'project_items': [2531, 202, 1195, 1230, 3726, 16019, 3732, 3766, 2589, 3743], 'train_complete': [2531, 202, 1195, 1230, 3726, 16019, 3732, 3766, 2589, 3743]}, 'bbbb': {'cv_done': 1, 'cv_progress': 100.0, 'pct_complete': 100.0, 'project_items': [2531, 202, 1195, 1230, 3726, 16019, 3732, 3766, 2589, 3743], 'train_complete': [2531, 202, 1195, 1230, 3726, 16019, 3732, 3766, 2589, 3743]}, 'cccc': {'cv_done': 1, 'cv_progress': 100.0, 'pct_complete': 100.0, 'project_items': [2531, 202, 1195, 1230, 3726, 16019, 3732, 3766, 2589, 3743], 'train_complete': [2531, 202, 1195, 1230, 3726, 16019, 3732, 3766, 2589, 3743]}}


### `/get_cv_results`

In [12]:
import requests
import json

url = 'https://fillet.azurewebsites.net/get_cv_results/'
# url = 'http://localhost:5000/get_cv_results/'


payload = {'project_id': 'cccc'}

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload,headers=headers,  timeout=5)
pd.DataFrame().from_dict((json.loads(r.json())))

,item_id,avg_sales,r2_score,mae_score,mpe_score,rmse_score
0,3743.0,132.480972,0.726140,30.217624,0.228090,39.697138
1,3732.0,126.142509,0.600136,26.932409,0.213508,34.499590
2,1195.0,129.492813,0.530031,30.880438,0.238472,40.411128
3,16019.0,109.613182,0.644549,27.317453,0.249217,36.303177
4,1230.0,8.996674,0.415926,4.199451,0.466778,5.721593
5,2531.0,318.983582,0.591923,69.529489,0.217972,90.647038
6,3726.0,74.383606,0.532206,17.621574,0.236901,23.117454
7,202.0,2110.440186,0.754612,312.160807,0.147913,428.454447
8,3766.0,91.448227,0.645032,20.341119,0.222433,27.470376
9,2589.0,161.616043,0.674297,39.336561,0.243395,55.437818


In [127]:
r.json()

{'status': 'incomplete'}

### `/optimize`

In [ ]:
url = 'http://localhost:5000/optimize/'

payload = {'project_id': 1,
           'constraints': [],
           'population': 100,
           'max_epoch': 200
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [ ]:
best_prices = r.json()['result']
best_prices

In [ ]:
import requests
import json

url = 'http://localhost:5000/predict/'
payload = {'prices': best_prices,
           'project_id': 1
           }
headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }
payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

# What If Prediction
qty = r.json()['qty_estimates']

In [ ]:
qty

In [ ]:
# Calculate Revenue
revenue = 0
for item in best_prices.keys():
    revenue += (best_prices[item]*qty['Qty_'+item])
    
print(revenue)

### `/predict/`

In [5]:
# Sample Actual Price 
from helper_functions import *

sales_data = data_subset

sales_data = optimize_memory(sales_data)

sales_data_wide = sales_data.set_index(
    ['Wk', 'Tier', 'Store', 'Item_ID']).unstack(level=-1).reset_index().copy()
sales_data_wide.columns = [
    ''.join(str(i) for i in col).strip()
    for col in sales_data_wide.columns.values
]
sales_data_wide = sales_data_wide.sort_values(
    ['Tier', 'Store', 'Wk'], ascending=True).reset_index(drop=True)

sales_data_wide_clean = sales_data_wide.dropna(axis=0).copy()
dataset = sales_data_wide_clean
price_columns = [
    col for col in sales_data_wide_clean.columns if col.startswith('Price')
]

a = dataset.sample(1)
a_dict = a[price_columns].iloc[0].to_dict()
a_input = {}
for key in a_dict.keys():
    a_input[key.split('_')[1]]=round(a_dict[key],2)

print('Input Prices Quantities:')
a_input




Input Prices Quantities:


{'202': 2.35,
 '1195': 3.15,
 '1230': 3.25,
 '2531': 6.25,
 '2589': 11.9,
 '3726': 9.95,
 '3732': 11.55,
 '3743': 11.1,
 '3766': 9.55,
 '16019': 13.6}

In [10]:
# Predict Sales Qty and Compare to Actual

import requests
import json

url = 'https://fillet.azurewebsites.net/predict/'
# url = 'http://localhost:5000/predict/'

payload = {'prices': a_input, 'project_id': 'cccc', 'modeltype': 'default'}

headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)
qty = r.json()['qty_estimates']
qty_outp = {}
for key in qty.keys():
    qty_outp[key] = round(qty[key], 0)
actual = a[[
    col for col in sales_data_wide_clean.columns if col.startswith('Qty_')
]].iloc[0].to_dict()
pd.DataFrame([qty_outp, actual], index=['Estimated', 'Actual']).transpose()

KeyError: 'qty_estimates'

In [11]:
r.json()

{'error': 'project not found'}

In [32]:
r.json()

{'qty_estimates': {'Qty_1230': 17,
  'Qty_16019': 205,
  'Qty_202': 5244,
  'Qty_3732': 344,
  'Qty_3766': 236}}

### `/get_feature_importances`

In [2]:
import requests
import json
import pandas as pd

url = 'https://fillet.azurewebsites.net/get_feature_importances/'
# url = 'http://localhost:5000/get_cv_results/'


payload = {'project_id': 'bbbb'}

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload,headers=headers,  timeout=240)
pd.DataFrame(r.json()['1230']).sort_values('importance', ascending=False)

,feature_name,importance
6,Price_3726,0.144873
3,Price_202,0.127241
9,Price_3766,0.116211
5,Price_2589,0.104554
8,Price_3743,0.101052
4,Price_2531,0.098390
7,Price_3732,0.088620
1,Price_1230,0.084403
2,Price_16019,0.071747
0,Price_1195,0.062908


In [6]:
import pprint

pprint.pprint(r.json())

{'1195': {'feature_name': {'0': 'Price_1195',
                           '1': 'Price_1230',
                           '2': 'Price_16019',
                           '3': 'Price_202',
                           '4': 'Price_2531',
                           '5': 'Price_2589',
                           '6': 'Price_3726',
                           '7': 'Price_3732',
                           '8': 'Price_3743',
                           '9': 'Price_3766'},
          'importance': {'0': 0.08310288190841675,
                         '1': 0.12035448849201202,
                         '2': 0.0791492611169815,
                         '3': 0.09970279783010483,
                         '4': 0.12043453752994537,
                         '5': 0.10218780487775803,
                         '6': 0.08286947011947632,
                         '7': 0.0973411276936531,
                         '8': 0.10491122305393219,
                         '9': 0.10994645208120346}},
 '1230': {'feature_name': {'0

### /detect_conflict


In [8]:
import requests
import json

url = 'https://fillet.azurewebsites.net/detect_conflict/'
with open('constraints_1.json') as f:
    constraints = json.load(f)

payload = {'constraints': constraints
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [9]:
r.json()

{'conflict': 'No conflict'}